In [1]:
from glob import glob
import pandas as pd

In [2]:
# Read in the most recent auxiliary data file available
aux_files = glob('/data/eop/compiled_country_data/auxiliary_data/auxiliary_data_*.csv')
file = max(aux_files, key=lambda x: x.split('_')[-1].split('.')[0])
accumulated_data = pd.read_csv(file)

In [3]:
survey_countries = pd.read_csv(
    '/data/eop/compiled_country_data/survey_countries.csv',
    dtype=str
)
accumulated_data = accumulated_data.merge(survey_countries[['country_code', 'pip_using']], on='country_code', how='left')

In [4]:
discrepancies = accumulated_data.copy()
discrepancies['discrepancy'] = (
    discrepancies.survey_poverty_rate_povertyline_2017
    - discrepancies.wb_poverty_rate_povertyline_2017_survey_year
)

discrepancies['abs_discrepancy'] = discrepancies.discrepancy.abs()
discrepancies.sort_values('abs_discrepancy', ascending=False, inplace=True)

In [5]:
discrepancies = discrepancies[
    ~discrepancies.country.isin(
        ['madagascar']
    )
]
discrepancies = discrepancies[
    [
        'country', 'survey_name', 'survey_poverty_rate_povertyline_2017', 
        'wb_poverty_rate_povertyline_2017_survey_year', 'discrepancy', 'abs_discrepancy', 'pip_using'
    ]
]

discrepancies_over_threshold = discrepancies[discrepancies.abs_discrepancy >= 0.04]


In [6]:
discrepancies[discrepancies.pip_using == 'True']

,country,survey_name,survey_poverty_rate_povertyline_2017,wb_poverty_rate_povertyline_2017_survey_year,discrepancy,abs_discrepancy,pip_using
126,rwanda,Integrated Household Living Conditions Survey ...,0.277411,0.241939,3.547276e-02,3.547276e-02,True
136,south africa,Income and Expenditure Survey 2010-2011,0.155183,0.180360,-2.517673e-02,2.517673e-02,True
8,bangladesh,Household Income and Expenditure Survey (HIES)...,0.050012,0.035685,1.432667e-02,1.432667e-02,True
67,indonesia,National Socio-Economic Survey (SUSENAS) 2018,0.054016,0.043021,1.099466e-02,1.099466e-02,True
60,guinea-bissau,Enquête Harmonisée sur le Conditions de Vie de...,0.210980,0.216605,-5.624938e-03,5.624938e-03,True
66,india,Household Consumption Expenditure Survey: 2022-23,0.028478,0.023492,4.985970e-03,4.985970e-03,True
77,kenya,Kenya Continuous Household Survey 2021,0.361461,0.361461,-2.082363e-08,2.082363e-08,True
20,burkina faso,Enquête Harmonisée sur le Conditions de Vie de...,0.312038,0.312038,1.262292e-08,1.262292e-08,True
93,mali,Enquête Harmonisée sur le Conditions de Vie de...,0.152188,0.152188,-5.879153e-09,5.879153e-09,True
111,niger,Enquête Harmonisée sur le Conditions de Vie de...,0.509059,0.509059,4.214056e-09,4.214056e-09,True


In [7]:
# All discrepancies
discrepancies.discrepancy.mean()

0.00180790410089952

In [8]:
# Just west african countries
discrepancies.loc[
    discrepancies.country.isin(
        ['burkina faso', 'benin', 'mali', 'niger', 'togo', "cote d'ivoire", 'guinea-bissau', 'senegal']
    ), 
    'discrepancy'
].mean()

-0.000803561129682733

In [9]:
# All, not including Ghana or South Sudan
discrepancies.loc[
    ~discrepancies.country.isin(
        ['ghana', 'south sudan']
    ), 
    'discrepancy'
].mean()

0.0018982992936663258